# Datasets are marked

In [10]:
import json
import pandas as pd

datafile = './panoptic_val2017.json'
datafile = './panoptic_train2017.json'

with open(datafile, 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data['annotations'])
df

,segments_info,file_name,image_id
0,"[{'id': 8345037, 'category_id': 51, 'iscrowd':...",000000000009.png,9
1,"[{'id': 6128041, 'category_id': 25, 'iscrowd':...",000000000025.png,25
2,"[{'id': 7045770, 'category_id': 64, 'iscrowd':...",000000000030.png,30
3,"[{'id': 5069153, 'category_id': 24, 'iscrowd':...",000000000034.png,34
4,"[{'id': 8159617, 'category_id': 1, 'iscrowd': ...",000000000036.png,36
...,...,...,...
118282,"[{'id': 4605256, 'category_id': 33, 'iscrowd':...",000000581906.png,581906
118283,"[{'id': 5067865, 'category_id': 9, 'iscrowd': ...",000000581909.png,581909
118284,"[{'id': 593189, 'category_id': 60, 'iscrowd': ...",000000581913.png,581913
118285,"[{'id': 8552070, 'category_id': 1, 'iscrowd': ...",000000581921.png,581921


In [12]:
import shutil
count = 0

for i in range(len(df)):
    if len(df["segments_info"][i]) >= 2 and len(df["segments_info"][i]) <= 6:
        dff = df["file_name"][i].split(".")[0] + ".jpg"
        # copy ./train2017/dff to ./few26
        shutil.copyfile(f"./train2017/{dff}", f"./few26/{dff}")

# Object Detection

In [ ]:
from ultralytics import YOLO
import cv2

img_path = "./CRV.png"
model = YOLO("yolov8n.pt")
img = cv2.imread(img_path)

results = model(img, conf=0.7)
nm = results[0].boxes.cls.cpu().numpy()

if len(nm) >= 2 and len(nm) <= 6:
    # Save the image


0: 448x640 1 person, 1 car, 14.8ms
Speed: 2.9ms preprocess, 14.8ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)
:
2


# VLM Detection

In [ ]:
import os
import shutil
import base64
import sys
from openai import OpenAI
import mimetypes

folder_path = './few26/'
dst_path = "./gpt-4o/"
os.makedirs(dst_path, exist_ok=True)

file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
detial_filename = [os.path.join(folder_path, f) for f in file_list]

client = OpenAI(api_key="your_api_key")
MODEL = "gpt-4o"

system_prompt = """
Please determine whether the image meets all of the following conditions. Reply with “Yes” or “No” only, no explanation needed:

1. The image contains 2 to 6 distinct and clearly identifiable main objects (e.g., people, animals, vehicles, furniture). Attached items like a backpack on a person count as part of that object.
2. The background is clean and uncluttered — it should not be a pure landscape, a beach crowded with people, or any visually messy scene.
3. The objects must be clearly distinguishable as separate entities.
"""

processed = 0
max_images = len(detial_filename)
for i, img_path in enumerate(detial_filename):
    if processed >= max_images:
        break

    mime_type, _ = mimetypes.guess_type(img_path)
    if mime_type not in ["image/jpeg", "image/png"]:
        print(f"Skipping unsupported mime type: {img_path}")
        continue

    try:
        with open(img_path, "rb") as img_file:
            img_data = img_file.read()
            base64_images = base64.b64encode(img_data).decode('utf-8')
    except Exception as e:
        print(f"Error reading image {img_path}: {e}")
        continue

    messages = [{
        "role": "user",
        "content": [
            {"type": "text", "text": system_prompt},
            {"type": "image_url", "image_url": {"url": f"data:{mime_type};base64,{base64_images}"}}
        ]
    }]

    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.0,
        )
        ai_message = response.choices[0].message.content.strip()
        print("Image path:", img_path, "----------", "gpt-4o-mini:", ai_message)

        if ai_message == "Yes":
            shutil.copy(img_path, os.path.join(dst_path, os.path.basename(img_path)))

        processed += 1

    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

sys.stdout.flush()